In [ ]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, _LRScheduler
from torch.optim import AdamW
import math
from rigl_torch.datasets import get_dataloaders


net = ModelFactory.load_model("wide_resnet22", "cifar10")

In [ ]:
from rigl_torch.datasets import get_dataloaders
from omegaconf import DictConfig
from rigl_torch.models import ModelFactory
import hydra

with hydra.initialize(config_path="../configs"):
    cfg = hydra.compose(config_name="config.yaml", overrides=[])
cfg

/tmp/ipykernel_171789/4275460388.py:6: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize(config_path="../configs"):


{'dataset': {'name': 'cifar10', 'normalize': False, 'num_classes': 10, 'classes': ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']}, 'model': {'name': 'wide_resnet22'}, 'experiment': {'comment': None, 'name': '${model.name}_${dataset.name}_${experiment.comment}_0.1LR'}, 'paths': {'data_folder': '/home/condensed-sparsity/data', 'artifacts': '/home/condensed-sparsity/artifacts', 'logs': '/home/condensed-sparsity/logs'}, 'rigl': {'dense_allocation': 0.1, 'delta': 100, 'grad_accumulation_n': 1, 'alpha': 0.3, 'static_topo': 0, 'const_fan_in': False, 'sparsity_distribution': 'erk', 'erk_power_scale': 1.0}, 'training': {'batch_size': 128, 'test_batch_size': 10, 'epochs': 250, 'lr': 0.1, 'init_lr': 1e-06, 'warm_up_steps': 5, 'gamma': 0.2, 'dry_run': False, 'seed': 1, 'log_interval': 10, 'save_model': True, 'weight_decay': 0, 'momentum': 0.9, 'optimizer': 'adadelta'}, 'compute': {'no_cuda': False, 'cuda_kwargs': {'num_workers': 1, 'pin_memory': True, 's

In [5]:
import torchvision
cifar = torchvision.datasets.CIFAR10(root="../data/")

In [7]:
len(cifar)

50000

In [2]:
train, test = get_dataloaders(cfg)

Files already downloaded and verified


In [ ]:
128*391

50048

In [ ]:
len(train)

In [ ]:
for x in train:
    break

In [ ]:
x[0].shape

In [ ]:
import torch
T_max= int(250*50000/128)
adamw1 = AdamW(net.parameters(), lr=0.001)
adamw2 = AdamW(net.parameters(), lr=0.001)
linear_step = torch.optim.lr_scheduler.StepLR(adamw2, step_size=30000, gamma=0.2)
cawr = CosineAnnealingWarmRestarts(optimizer=adamw1, T_0=20, T_mult=2)
# ca = CosineAnnealingLR(optimizer=adamw2, T_max=T_max)
# adamw3 = AdamW(net.parameters(), lr=0.001)
# test= CosineAnnealingWithLinearWarmUp(optimizer=adamw3, T_max=T_max, warm_up_steps=50, lr=0.001)

In [ ]:
linear_step.get_last_lr()

In [ ]:
%matplotlib inline
warm_restart_lrs = []
linear_step_lrs = []
combined =[]
linear_warmup = []
for x in range(T_max):
    lr1 = cawr.get_last_lr()
    warm_restart_lrs.append(lr1)
    cawr.step()
    lr2 = linear_step.get_last_lr()
    linear_step_lrs.append(lr2)
    linear_step.step()
    # combined.append(lr1[0] * lr2[0]*1000)
    # lr3 = test.get_last_lr()
    # linear_warmup.append(lr3)
    # test.step()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(warm_restart_lrs)
plt.plot(linear_step_lrs)
# plt.plot(combined)
# plt.plot(linear_warmup)

In [ ]:
from rigl_torch.models import get_model
from rigl_torch.datasets import get_dataloaders
from omegaconf import DictConfig
import hydra

In [ ]:
with hydra.initialize(config_path="../configs"):
    cfg = hydra.compose(config_name="config.yaml", overrides=[])
cfg

In [ ]:
def print_image_shape(module, input, output):
    print(f"Module {module._get_name()}")
    print(f"input shape: {input[0].shape} output shape: {output.shape}")
    return output